In [ ]:
# import sys
import pandas as pd
from IPython.core.display_functions import display
# from IPython.core.display import HTML

In [ ]:
def to_dollars(df, fields):
    # Clean up dollar columns
    for x in fields:
        def f(x):
            try:
                return float(x)
            except ValueError:
                return float('nan')

        df[x] = df[x].replace('[\$,]', '', regex=True).apply(f).values

def to_percent(df, fields):
    # Clean up percentage columns
    for x in fields:
        def f(x):
            try:
                return float(x)
            except ValueError:
                return float('nan')

        df[x] = df[x].replace('[%,]', '', regex=True).apply(f).values

fn = 'Portfolio_Positions_Jul-30-2022.csv'

df = pd.read_csv(fn)
df = df.dropna(subset=['Account Name']).reset_index()
df = df.drop(columns=['index'])

# pending = df[df['Symbol'] == 'Pending Activity'][['Account Number', 'Account Name', 'Current Value']]
# to_dollars(pending, ['Current Value'])
# pending.rename(columns = {'Current Value': 'Pending'}, inplace=True)

to_dollars(df, ['Current Value', 'Last Price', 'Cost Basis', 'Cost Basis Per Share', "Today's Gain/Loss Dollar",
                'Last Price Change', 'Total Gain/Loss Dollar'])

to_percent(df, ["Today's Gain/Loss Percent", "Total Gain/Loss Percent", "Percent Of Account"])

df = df.replace(r'^n/a$', float('nan'), regex=True)

# with pd.option_context('display.max_rows', None, 'display.max_columns', None, "expand_frame_repr", True):
#     display(df)

if True:
    # Combine lots of the same security that are held in Margin and Cash
    df = df.drop(columns=['Type'])

    f = dict.fromkeys(df, 'first')
    # f.update(dict.fromkeys(df.columns[df.dtypes.eq(object)], 'first'))
    for i in ['Quantity', 'Current Value', 'Percent Of Account', 'Cost Basis', "Today's Gain/Loss Dollar",
              'Total Gain/Loss Dollar']:
        f[i] = 'sum'

    del f['Account Number']
    del f['Symbol']
    df = df.groupby(['Account Number', 'Symbol'], as_index=False).agg(f)

    # These are incorrect after the grouping. The correct result requires calculation of a mean weighted by Quantity.
    # Probably best to only keep primary columns and recalculate derived columns.
    #   "Today's Gain/Loss Percent": 'first'
    #   'Total Gain/Loss Percent': 'first'
    #   'Cost Basis Per Share': 'first'

    df["Today's Gain/Loss Percent"] = df["Today's Gain/Loss Dollar"] / df['Current Value']
    df['Cost Basis Per Share'] = df['Cost Basis'] / df['Quantity']

# with pd.option_context('display.max_rows', None, 'display.max_columns', None, "expand_frame_repr", True):
#     display(df)

In [ ]:
# https://ipython.readthedocs.io/en/stable/api/generated/IPython.core.interactiveshell.html
# %config ZMQInteractiveShell
# %config ZMQInteractiveShell.ast_node_interactivity='last_expr_or_assign'

In [ ]:
account_totals = df[['Account Name', 'Symbol', 'Current Value']].pivot(index='Symbol', columns='Account Name',
                                                                       values='Current Value').sum()

all_symbols = df['Symbol'].unique()
bonds = df[df['Symbol'].str.contains('^[0-9]{3}[a-zA-Z0-9]{2}[a-zA-Z0-9*@#]{3}[0-9](?![0-9a-zA-Z-])')][
    'Symbol'].unique()
cash = df[df['Symbol'].str.contains('\*\*$') + (df['Symbol'] == 'Pending Activity')]['Symbol'].unique()
pending = df[~df['Symbol'].isin(bonds) & ~df['Symbol'].isin(cash)]['Symbol'].unique()
funds_stocks_options = df[~df['Symbol'].isin(bonds) & ~df['Symbol'].isin(cash)]['Symbol'].unique()
leftover = list(set(all_symbols) - set(funds_stocks_options) - set(bonds) - set(cash))
if leftover:
    print('Warning: Leftover symbols')
    display(leftover)

cash_ = df[df['Symbol'].isin(cash)]
cash_totals = cash_[['Account Name', 'Symbol', 'Current Value']].pivot(index='Symbol', columns='Account Name',
                                                                       values='Current Value').sum()
# Adjust cash totals by amount pending
# cash_totals.rename("Cash", inplace=True)
# df2 = pd.merge(cash_totals, pending, how='left', left_on=['Account Name'], right_on=['Account Name']).fillna(0.)
# df2['Cash'] += df2['Pending']
# cash_totals = df2[['Account Name', 'Cash']].rename(columns={'Cash': 'Cash Total'})
display(cash_totals)

In [ ]:
fso = df[df['Symbol'].isin(funds_stocks_options)]
fso_totals = fso[['Account Name', 'Symbol', 'Current Value']].pivot(index='Symbol', columns='Account Name',
                                                                    values='Current Value').sum()
fso_pt = fso[['Account Name', 'Symbol', 'Current Value']].pivot(index='Account Name', columns='Symbol',
                                                                values='Current Value')

fso_pt_ = fso_pt.copy()
fso_pt_['Eq Total'] = fso_pt_.sum(axis=1)
eq_total = fso_pt_['Eq Total']
fso_pt_['Total'] = account_totals
fso_pt_['Eq Per'] = fso_pt_['Eq Total'] / fso_pt_['Total'] * 100
fso_pt_.loc['Total'] = fso_pt_.sum(numeric_only=True, axis=0)
with pd.option_context('display.max_rows', None, 'display.max_columns', None, "expand_frame_repr", False,
                       "display.float_format", '${:,.2f}'.format):
    display(fso_pt_.fillna('.'))

fso_pt_ = fso_pt.copy()
fso_pt_.loc['Total'] = fso_pt_.sum(numeric_only=True, axis=0)
fso_pt_ = fso_pt_.div(fso_pt_.sum(axis=1), axis=0)
with pd.option_context('display.max_rows', None, 'display.max_columns', None, "expand_frame_repr", False,
                       "display.float_format", '{:.2%}'.format):
    display(fso_pt_.fillna('.'))

In [ ]:
fso = df[df['Symbol'].isin(bonds)]
fso_totals = fso[['Account Name', 'Symbol', 'Current Value']].pivot(index='Symbol', columns='Account Name',
                                                                    values='Current Value').sum()
fso_pt = fso[['Account Name', 'Symbol', 'Current Value']].pivot(index='Account Name', columns='Symbol',
                                                                values='Current Value')

fso_pt_ = fso_pt.copy()
fso_pt_['Bond Total'] = fso_pt_.sum(axis=1)
bond_totals = fso_pt_['Bond Total']
fso_pt_['Total'] = account_totals
fso_pt_['Bond Per'] = fso_pt_['Bond Total'] / fso_pt_['Total'] * 100
fso_pt_.loc['Total'] = fso_pt_.sum(numeric_only=True, axis=0)
with pd.option_context('display.max_rows', None, 'display.max_columns', None, "expand_frame_repr", False,
                       "display.float_format", '${:,.2f}'.format):
    display(fso_pt_.fillna('.').transpose())

fso_pt_ = fso_pt.copy()
fso_pt_.loc['Total'] = fso_pt_.sum(numeric_only=True, axis=0)
fso_pt_ = fso_pt_.div(fso_pt_.sum(axis=1), axis=0)
with pd.option_context('display.max_rows', None, 'display.max_columns', None, "expand_frame_repr", False,
                       "display.float_format", '{:.2%}'.format):
    display(fso_pt_.fillna('.').transpose())

In [ ]:
cash_totals.name = 'Cash Total'
account_totals.name = 'Total'
summary = pd.concat([cash_totals, bond_totals, eq_total, account_totals], axis=1)
f = lambda x: '${:,.2f}'.format(x) if not pd.isnull(x) else x
summary['Cash Total'] = summary['Cash Total'].apply(f)
summary['Bond Total'] = summary['Bond Total'].apply(f)
summary['Eq Total'] = summary['Eq Total'].apply(f)
summary['Total'] = summary['Total'].apply(f)

summary2 = pd.concat([cash_totals, bond_totals, eq_total], axis=1)
summary2 = summary2.div(summary2.sum(axis=1), axis=0)
summary2 = summary2.rename(columns={'Cash Total': 'Cash%', 'Bond Total': 'Bond%', 'Eq Total': 'Eq%'})
f = lambda x: '{:.1%}'.format(x) if not pd.isnull(x) else x
summary2['Cash%'] = summary2['Cash%'].apply(f)
summary2['Bond%'] = summary2['Bond%'].apply(f)
summary2['Eq%'] = summary2['Eq%'].apply(f)

summary = pd.concat([summary2, summary], axis=1).fillna('.')

with pd.option_context('display.max_rows', None, 'display.max_columns', None, "expand_frame_repr", False):
    display(summary)
